In [1]:
import geemap,ee
import pandas as pd
import os

ee.Initialize()

path = r'C:\Users\xiong\Documents\geemap\shp\jxwgs'+os.sep+'jx.shp'
ee_shp = geemap.shp_to_ee(path)
jx = ee_shp.geometry()

path = r'C:/Users/xiong/Documents/geemap/shp/2m/vi-wi'+os.sep+'point1.shp'
sample = geemap.shp_to_ee(path)

print(sample.size().getInfo())
Map = geemap.Map()
# Map

37


In [2]:
# NDWI LSWI mNDWI NMDI

def cloudfunction_ST2(image):
    # use add the cloud likelihood band to the image
    qa = image.select("QA60")
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

    # Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask).divide(10000)\
        .select("B.*")\
        .copyProperties(image, ["system:index","system:time_start"])

def NDWI(img):
    ndwi = img.normalizedDifference(['B3', 'B8']).rename('NDWI')
    return ndwi

def NDVI(img):
    ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return ndvi

def mNDWI(img):
    mndwi = img.normalizedDifference(['B3', 'B11']).rename('mNDWI')
    return mndwi
    
def LSWI(img):
    lswi = img.normalizedDifference(['B8', 'B11']).rename('LSWI')
    return lswi

def NMDI(img):
    nwdi = img.expression(
        '(NIR-(SWIR1-SWIR2))/(NIR+(SWIR1-SWIR2))', {
          'NIR': img.select('B8'),
          'SWIR1': img.select('B11'),
          'SWIR2':img.select('B12')}).rename('NMDI')
    return nwdi

def EVI(img):
    nir = img.select("B8")
    red = img.select("B4")
    blue = img.select("B2")
    evi = img.expression("2.5*(B4 - B3)/(B4 + 6*B3 - 7.5*B1 + 1)",{"B4": nir,"B3": red ,"B1": blue})
    evi = evi.rename('EVI')
    return evi

# 获取ps点的img数据，col_names为要筛选的波段数据
def sample_points_to_df(img, ps, col_names=None):
    ps_data = img.sampleRegions(ps, 
                                geometries=True, 
                                scale=ee.ImageCollection('COPERNICUS/S2').first().select(0).projection().nominalScale())
    values_list = [f['properties'] for f in ps_data.getInfo()["features"]]
    if col_names == None:
        values_df = pd.DataFrame(values_list)
    else:
        values_df = pd.DataFrame(values_list)[col_names]
    return values_df

In [3]:
bands = ['NDVI','NDWI','mNDWI','LSWI','NMDI','EVI']

for month in range(3,11):
    print(month,end='-')
    outfile = r'C:\Users\xiong\Downloads\vi-wi'+os.sep+str(month)+'.csv'
    
#     !jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000 
    
    if os.path.exists(outfile):
        continue
    
    s2m = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
            .filter(ee.Filter.calendarRange(2017,2021,'year')).filter(ee.Filter.calendarRange(month,month+1,'month'))\
            .filterBounds(jx)\
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',25))\
            .sort('CLOUDY_PIXEL_PERCENTAGE', False)\
            .map(cloudfunction_ST2)\
            .mosaic()

    s2m = s2m.addBands(NDWI(s2m))
    s2m = s2m.addBands(NDVI(s2m))
    s2m = s2m.addBands(mNDWI(s2m))
    s2m = s2m.addBands(LSWI(s2m))
    s2m = s2m.addBands(NMDI(s2m))
    s2m = s2m.addBands(EVI(s2m))
    
#     r_shp = s2m.select(bands).sampleRegions(collection=sample,
#                               scale=ee.ImageCollection('COPERNICUS/S2').first().select(0).projection().nominalScale().getInfo(), 
#                               geometries=True)
    
#     arr = r_shp.reduceColumns(ee.Reducer.toList(len(bands)), bands).values().get(0).getInfo()
#     df = pd.DataFrame(arr,columns=bands)
    
    df = sample_points_to_df(s2m,sample,bands)

    df.to_csv(outfile,index=False,header=True)
#     print(r_shp.size().getInfo())

3-4-5-6-7-8-9-10-